In [1]:
# -- run me first --
from pprint import pprint  # for pretty printing
# display all outputs, not only last one
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
print("-done-")

-done-


<center>🐍</center>

***
# 7. Генераторы и Итераторы
<div style="text-align: right; font-weight: bold">Aleksandr Koriagin</div>
<div style="text-align: right; font-weight: bold"><span style="color: #76CDD8;">&lt;</span>epam<span style="color: #76CDD8;">&gt;</span></div>
<div style="text-align: right; font-weight: bold">May 2020</div>
<div style="text-align: right; font-style: italic">Nizhny Novgorod</div>

***
## Оглавление<a id="0"></a>

1. [Определения](#1)
2. [Контейнеры](#2)
3. [Итерируемые объекты](#3)
4. [Итераторы](#4)
5. [Генераторы](#5)
    1. Типы генераторов
6. [Заключение](#6)
7. [Дополнения](#7)
    1. Множественные `yield`
    1. Исключения в генераторах
8. [Расширенный синтаксис генераторов](#8)
    1. Использование yield как выражения / Сопрограммы / Coroutines
    1. Использование return в генераторах
    1. Использование yield from
9. [Домашняя работа](#9)

In [ ]:
%%bash
# generate table of contents
cat 7_generators_iterators.ipynb | grep "##" | grep -v "cat" | sed  "s/#/    /g" | tr -d '"'

***
## 1. Определения<a id="1"></a>

Рано или поздно всякий Python-разработчик сталкивается с осознанием того, что он не до конца понимает смысл и различия следующих понятий:

* контейнер (container)
* итерируемый объект (iterable)
* итератор (iterator)
* генератор (generator)
* генераторное выражение (generator expression)
* списочное включение (list comprehension)

Для начала будет неплохо ознакомиться с небольшой шпаргалкой, которая в будущем может пригодится, если вдруг снова забудете что к чему.

![reference](7_generators_iterators_files/relationships.png "Reference")

## 2. Контейнеры<a id="2"></a>

Контейнер — это тип данных, предназначенный для хранения элементов и предоставляющий набор операций для работы с ними. Сами контейнеры и, как правило, их элементы хранятся в памяти. В Python существует масса разнообразных контейнеров, среди которых всем хорошо знакомые:

* `list`, `deque`, …
* `set`, `frozensets`, …
* `dict`, `defaultdict`, `OrderedDict`, `Counter`, …
* `tuple`, `namedtuple`, …
* `str`

Контейнеры легко понять, проведя аналогию с реальным миром: контейнер можно рассматривать как некую коробку, в которой хранятся вещи.

Технически объект является контейнером тогда, когда он предоставляет возможность определить наличие или отсутствие в нём конкретного элемента. Например, проверка вхождения элемента в список, кортеж или множество выполняется следующим образом:

In [ ]:
1 in     [1, 2, 3]      # списки
4 not in [1, 2, 3]
1 in     {1, 2, 3}      # множества
4 not in {1, 2, 3}
1 in     (1, 2, 3)      # кортежи
4 not in (1, 2, 3)

Применительно к словарям операция проверки вхождения работает с ключами словаря:

In [ ]:
d = {1: 'foo', 2: 'bar', 3: 'qux'}
1 in d
4 not in d
'foo' not in d  # 'foo' не является _ключом_ словаря

Ну и, наконец, подобным образом вы можете выполнять проверку вхождения подстроки в строку:

In [ ]:
s = 'foobar'
'b' in s
'x' not in s
'foo' in s

Последний пример выглядит немного странно, но он является хорошим примером применения интерфейса контейнеров. Хотя строка в буквальном смысле и не хранит все возможные подстроки в памяти, тем не менее, благодаря контейнерному интерфейсу мы имеем возможность выполнять поиск подстрок подобным образом.

Обратите внимание, что несмотря на то, что большинство контейнеров предоставляют возможность извлекать из них любой элемент, само по себе наличие этой возможности не делает объект контейнером, а лишь **итерируемым объектом**, о чём будет рассказано дальше. Также, контейнер не обязан быть итерируемым. Например, [фильтр Блума](https://ru.wikipedia.org/wiki/%D0%A4%D0%B8%D0%BB%D1%8C%D1%82%D1%80_%D0%91%D0%BB%D1%83%D0%BC%D0%B0) предоставляет возможность узнать, содержится ли элемент в структуре данных, но при этом нет возможности извлекать из неё отдельные элементы.

## 3. Итерируемые объекты<a id="3"></a>

Как было уже сказано, большинство контейнеров являются итерируемыми. Но в то же время множество других типов данных являются итерируемыми, например, файлы, сокеты и тому подобные. В то время как контейнеры обычно содержат конечное количество элементов, просто итерируемый объект может представлять источник бесконечных данных.

**Итерируемым объектом** является любой объект, который может предоставить **итератор**, который, в свою очередь, и возвращает отдельные элементы. На первый взгляд это звучит немного странновато, но тем не менее очень важно понимать разницу между интерируемым объектом и итератором. Рассмотрим пример:

In [ ]:
x = [1, 2, 3]
y = iter(x)
z = iter(x)

next(y)
next(y)
next(z)

type(x)
type(y)

Здесть `х` — это итерируемый объект, в то время как `y` и `z` два отдельных экземпляра итератора, производящего значения из итерируемого объекта `x`. Как видим, `y` и `z` сохраняют состояние между вызовами `next()`. В данном примере в качестве источника данных для итератора используется список, но это не является обязательным условием.

Часто, чтобы сократить объем кода, классы итерируемых объектов имплементируют сразу оба метода: `__iter__()` и `__next__()`, при этом `__iter__()` возвращает `self`. Таким образом класс одновременно является и итерируемым и итератором самого себя. Однако, лучшей практикой всё же считается в качестве итератора возвращать отдельный объект.

Итак, когда выполняется следующий код:
```python
x = [1, 2, 3]
for elem in x:
    pass
```
вот что на самом деле происходит:
![iterable-iterator](7_generators_iterators_files/iterable-vs-iterator.png "iterable-iterator")

Если диассемблировать код, представленный выше, мы обнаружим вызов `GET_ITER`, который по сути является следствием вызова `iter(x)`. Инструкция `FOR_ITER` является эквивалентом многократного вызова `next()` до тех пор, пока не будет возвращён последний элемент. Этого, правда, не видно в байт-коде из-за оптимизаций, вносимых интерпретатором.

In [2]:
import dis
x = [1, 2, 3]
dis.dis('for _num in x: pass')

  1           0 SETUP_LOOP              12 (to 14)
              2 LOAD_NAME                0 (x)
              4 GET_ITER
        >>    6 FOR_ITER                 4 (to 12)
              8 STORE_NAME               1 (_num)
             10 JUMP_ABSOLUTE            6
        >>   12 POP_BLOCK
        >>   14 LOAD_CONST               0 (None)
             16 RETURN_VALUE


## 4. Итераторы<a id="4"></a>

Итак, чем же является итератор?
<br>Итератор — это вспомогательный объект, возвращающий следующий элемент всякий раз, когда выполняется вызов функции `next()` с передачей этого объекта в качестве аргумента. Таким образом, любой объект, предоставляющий метод `__next__()`, является итератором, возвращающим следующий элемент при вызове этого метода, при этом совершенно неважно как именно это происходит.

Итак, итератор — это некая фабрика по производству значений. Всякий раз, когда вы обращаетесь к ней с просьбой "давай следующее значение", она знает как сделать это, поскольку сохраняет своё внутреннее состояние между обращениями к ней.

Существует бесчисленное множество примеров использования итераторов. Например, все функции пакета [`itertools`](https://docs.python.org/3/library/itertools.html) возвращают итераторы. 

Некоторые из которых генерируют бесконечные последовательности:

In [ ]:
from itertools import count
counter = count(start=13)
next(counter)
next(counter)

Некоторые создают бесконечные последовательности из конечных:

In [4]:
from itertools import cycle
colors = cycle(['red', 'white', 'blue'])

iter(colors)
next(colors)
next(colors)
next(colors)
next(colors)

'red'

'white'

'blue'

'red'

Или конечные последовательности из бесконечных:

In [ ]:
from itertools import islice
colors = cycle(['red', 'white', 'blue'])  # infinite
limited = islice(colors, 0, 4)            # finite
for x in limited:                         # so safe to use for-loop on
    print(x)

Чтобы лучше понять внутреннюю логику работы итераторов, давайте самостоятельно сделаем **итератор**, который будет генерировать последовательность [чисел Фибоначчи](https://ru.wikipedia.org/wiki/%D0%A7%D0%B8%D1%81%D0%BB%D0%B0_%D0%A4%D0%B8%D0%B1%D0%BE%D0%BD%D0%B0%D1%87%D1%87%D0%B8).

In [5]:
from itertools import islice
class Fib:
    def __init__(self):
        self.prev = 0
        self.curr = 1

    def __iter__(self):
        return self

    def __next__(self):
        value = self.curr
        self.curr += self.prev
        self.prev = value
        return value

fib = Fib()
list(islice(fib, 0, 15))

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610]

In [10]:
class FibFinit:
    def __init__(self, stop=650):
        self.prev = 0
        self.curr = 1
        self.stop = stop

    def __iter__(self):  # try to disable
        return self

#     def __next__(self):
#         if self.curr > self.stop:
#             raise StopIteration
#         value = self.curr
#         self.curr += self.prev
#         self.prev = value
#         return value

fib = FibFinit()
list(fib)
iter(fib)

# fib = FibFinit()
# next(fib)
# next(fib)
# next(fib)

TypeError: iter() returned non-iterator of type 'FibFinit'

Обратите внимание, что приведённый выше класс одновременно является **итерируемым объектом** (поскольку реализует метод `__iter__()`) и **итератором** (поскольку реализует метод `__next__()`).

Состояние этого итератора полностью хранится в переменных `prev` и `curr` экземпляра объекта, значение которых используется при каждом последующем обращении к итератору.

Каждый вызов `next()` делает две важные вещи:
1. модифицирует внутреннее состояние объекта;
2. возвращает следующий элемент последовательности.

Снаружи итератор напоминает фабрику по производству значений, которая "спит" до тех пор, пока у неё не попросят выдать очередное значение, после возврата которого фабрика снова "засыпает".

## 5. Генераторы<a id="5"></a>

Итак, наконец-то мы добрались до самого интересного! Генераторы являются безумно интересной и полезной штукой в Python. Генератор — это особый, более изящный случай итератора.

Используя генератор, вы можете создавать итераторы, вроде того, что мы рассмотрели выше, используя более лаконичный синтаксис, и не создавая при этом отдельный класс с методами `__iter__()` и `__next__()`.

Давайте внесём немного ясности:
* любой генератор является итератором (но не наоборот!);
* следовательно, любой генератор является "ленивой фабрикой", возвращающей значения последовательности по требованию.

Вот пример итератора последовательности чисел Фибоначчи в исполнении генератора:

In [ ]:
def fib():
    prev, curr = 0, 1
    while True:
        yield curr
        prev, curr = curr, prev + curr

f = fib()
list(islice(f, 0, 15))

Ну как? Не находите, что это выглядит намного элегантнее простого итератора? Весь секрет кроется в ключевом слове `yield`. Давайте разберёмся, что к чему.

Во-первых, обратите внимание, что `fib` является обычной функцией, ничего особенного. Однако же, в ней отсутствует оператор `return`, возвращающий значение. В данном случае возвращаемым значением функции будет генератор (то есть, по сути, итератор — фабрика значений, сохраняющая состояние между обращениями к ней).

Теперь, когда происходит вызов функции `fib`

In [ ]:
fib()

будет создан и возвращён экземпляр генератора. К данному моменту ещё никакого кода внутри функции не выполняется и генератор ожидает вызова.

Дальше созданный экземпляр генератора передаётся в качестве аргумента функции `islice`, которая также возвращает итератор, следовательно также никакого кода функции `fib` пока что не выполняется.

И, наконец, происходит вызов `list()` с передачей в качестве аргумента итератора, возвращённого функцией `islice()`. Чтобы `list()` смогла построить объект списка на основе полученного аргумента, ей необходимо получить все значения из этого аргумента. Для этого `list()` выполняет последовательные вызовы метода `next()` итератора, возвращённого вызовом `islice()`, который, в свою очередь, выполняет последовательные вызовы `next()` в экземпляре итератора `f`.

При первом запросе значения итератора `f` будет выполнен код:
```python
prev, curr = 0, 1
```
После чего произойдёт вход в тело цикла и оператор `yield` вернёт первое значение. На этом работа кода внутри `f` будет приостановлена до следующего вызова `next()`. Значение, возвращенное при помощи `yield`, будет передано итератору `islice()`, который передаст его в `list()`, таким образом в список добавится значение.

При втором и последующих обращениях к итератору `f` будет выполняться код с того места, где было прервано выполнение, то есть после вызова оператора `yield`. После того, как этот код будет выполнен, выполнение цикла начнётся заново и так же, как и при первом вызове, итератор будет возвращать очередное значение и "засыпать" в аккурат после вызова `yield`.

Весь описанный процесс повторится 15 раз, до тех пор, пока `list()` не запросит у `islice()` 16й элемент, на что тот ответит выбросом исключения `StopIteration`, что будет свидетельствовать о том, что достигнут конец последовательности. Обратите внимание, что 16й элемент не будет сохранён в конечном объекте списка и память, отведённая под него, будет освобождена сборщиком мусора.

### Типы генераторов

В Python существует два типа генераторов: 
* генераторные функции
* генераторные выражения

Генератором является любая функция, содержащая `yield` в любом месте её кода. Пример такого генератора мы только что рассмотрели. 
<br>Другой разновидностью генераторов в Python являются генераторные выражения, своим видом напоминающие списочные выражения. Использование генераторных выражений бывает очень хорошим решением в ряде случаев.

Предположим, вы используете следующую конструкцию, чтобы создать список квадратов чисел:

In [ ]:
numbers = [1, 2, 3, 4, 5, 6]

[x * x for x in numbers]     # list comprehension
{x * x for x in numbers}     # set comprehension
{x: x * x for x in numbers}  # dict comprehension

Или, используя генераторное выражение (обратите внимание, это НЕ кортеж!):

In [11]:
numbers = [1, 2, 3, 4, 5, 6]

lazy_squares = (x * x for x in numbers)
lazy_squares
type(lazy_squares)

next(lazy_squares)
next(lazy_squares)
list(lazy_squares)

<generator object <genexpr> at 0x0000019E31C21648>

generator

1

4

[9, 16, 25, 36]

## 6. Заключение<a id="6"></a>

Генераторы являются потрясающей языковой конструкцией. Они позволяют писать код, используя меньше промежуточных переменных, снизить потребление памяти и ресурсов процессора, а также уменьшить объём самого кода. Если вы всё ещё не используете генераторы и хотели бы начать делать это, попробуйте начать с того, что обратите внимание на все участки вашего кода, имеющие вид:
```python
def something():
    result = []
    for ... in ...:
        result.append(x)
    return result
```
И замените их генераторами:
```python
def iter_something():
    for ... in ...:
        yield x
```

**Дополнительно**: [Итераторы и генераторы в Python](https://shepetko.com/ru/blog/python-iterable-iterators-generators); Original: [Iterables vs. Iterators vs. Generators](https://nvie.com/posts/iterators-vs-generators/)

## 7. Дополнения<a id="7"></a>

### Множественные `yield`

In [12]:
def gen(num):
    yield f"Yielding x2: {num * 2}"
    yield f"Yielding x3: {num * 3}"
    yield f"Yielding x4: {num * 4}"

f = gen(5)
list(f)

f = gen(5)
next(f)
next(f)
next(f)
next(f)

['Yielding x2: 10', 'Yielding x3: 15', 'Yielding x4: 20']

'Yielding x2: 10'

'Yielding x3: 15'

'Yielding x4: 20'

StopIteration: 

### Исключения в генераторах

Исключение `GeneratorExit` выбрасывается в месте вызова `yield` при удалении объекта генератора или вызове метода `.close()`. Это можно использовать для освобождения ресурсов.

Выбросить произвольное исключение в месте вызова `yield` можно с помощью метода `.throw()`

In [ ]:
def generate():
    for value in [1, 2, 3]:
        try:
            yield value
        except GeneratorExit:
            print('Caught GeneratorExit, re-raising')
            raise

my_generator = generate()
next(my_generator)
my_generator.close()  # del my_generator

***
## 8. Расширенный синтаксис генераторов<a id="8"></a>

> далее текст больше для знакомства, что такое есть в Python

### Использование yield как выражения / Сопрограммы / Coroutines

Генератор может не только возвращать значения, но и принимать их на вход.

Напишем простую реализацию генератора, который может складывать два аргумента, хранить историю результатов и выводить историю.

In [13]:
def calc():
    history = []
    while True:
        x, y = (yield)
        if x == 'h':
            print(f"history: {history}")
            continue
        result = x + y
        print(f"result: {result}")
        history.append(result)

c = calc()
type(c)  # <type 'generator'>

next(c)  # Необходимая инициация. Можно написать c.send(None)

c.send([1, 2])
c.send([100, 30])
c.send([666, 0])
c.send(["h", 0])
c.close()          # Закрывем генератор

generator

result: 3
result: 130
result: 666
history: [3, 130, 666]


Т.е. мы создали генератор, проинициализировали его и подаём ему входные данные.
<br>Он, в свою очередь, эти данные обрабатывает и ***сохраняет своё состояние между вызовами до тех пор пока мы его не закрыли***. После каждого вызова генератор возвращает управление туда, откуда его вызвали. 

In [14]:
def multi_yield():
    vals = ("a", "b", "c", "d", "e", "f")
    for i in vals:
        IN = yield i
        yield f"IN={IN}, i={i}"

m = multi_yield()

print(f"1) {next(m)}       ")
print(f"2) {next(m)}       ")
print(f"3) {m.send('AAA')} ")
print(f"4) {m.send('BBB')} ")
print(f"5) {next(m)}       ")
print(f"6) {next(m)}       ")

1) a       
2) IN=None, i=a       
3) b 
4) IN=BBB, i=b 
5) c       
6) IN=None, i=c       


In [15]:
# Больше примеров
def gen():
    for i in range(1, 10):
        X = yield i
        print(f"X={X}, i={i}")

G = gen()

print(f"1) next(G)    ==> {next(G)}    \n\n")

print(f"2) G.send(77) ==> {G.send(77)} \n\n")
print(f"3) G.send(88) ==> {G.send(88)} \n\n")

print(f"4) next(G)    ==> {next(G)}    \n\n")

1) next(G)    ==> 1    


X=77, i=1
2) G.send(77) ==> 2 


X=88, i=2
3) G.send(88) ==> 3 


X=None, i=3
4) next(G)    ==> 4    




In [16]:
# Больше примеров
def gen_with_reset(start, stop):
    value = start
    while value < stop:
        sent = (yield value)  # yield принимает данные из вызывающего его кода
        if sent is not None:  # If value provided, change counter
            value = sent
        else:
            value = value + 1

g = gen_with_reset(10, 15)
next(g)  # Необходимая инициация. Можно написать c.send(None)
next(g)

g
g.send(5)
next(g)
list(g)

10

11

<generator object gen_with_reset at 0x0000019E31C21948>

5

6

[7, 8, 9, 10, 11, 12, 13, 14]

### Использование return в генераторах

`return <something>` in a generator is now equivalent to `raise StopIteration(<something>)`

In [17]:
def gen_with_return(start):
    value = start
    while True:
        if value > start + 1:
            return "Enough, I'm sick and tired of this."
        yield value
        value += 1

list(gen_with_return(10))

g = gen_with_return(10)
next(g)
next(g)
next(g)

[10, 11]

10

11

StopIteration: Enough, I'm sick and tired of this.

### Использование yield from

For basic purposes we can use plain generators to compute values and to pass those values around. 
The benefits of `yield from` should become clear when we know what it does and in which situations it can be used.

Consider a generator that looks like this:

In [ ]:
def generator():
    for i in range(0, 10):
        yield i
    for j in range(10, 20):
        yield j

list(generator())

Как и ожидалось, генератор выдает числа от 0 до 19.
Допустим теперь мы хотим разделить этот генератор на два разных чтобы можно было использовать их отдельно.

In [19]:
def generator2():
    for i in range(0, 10):
        yield i

def generator3():
    for j in range(10, 20):
        yield j

def generator():
    for i in generator2():
        yield i
    for j in generator3():
        yield j

list(generator())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

Эта версия `generator()` тоже выдает числа от 0 до 19.
Но, возможно, это можно сделать проще?

In [ ]:
def generator():
    yield from generator2()
    yield from generator3()

list(generator())    

Такой подход дает нам такой же результат и делает функцию более понятной.

In [21]:
# еще можно использовать itertools.chain
from itertools import chain

def generator():
    yield from chain(generator2(), generator3())

list(generator())        

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

Другой пример

In [ ]:
def reader():
    for i in range(0, 4):
        yield f'reader: {i}'

def reader_wrapper(g):
    for i in g:
        yield i

wrap = reader_wrapper(reader())
list(wrap)

Вместо итерирования по `reader()`, мы можем просто сделать `yield from` из него

In [ ]:
def reader():
    for i in range(0, 4):
        yield f'reader: {i}'

def reader_wrapper2(g):
    yield from g

list(reader_wrapper2(reader()))

***
## 9. Домашняя работа<a id="9"></a>

### HW 1.

Создайте свой аналог `zip` с точно таким же поведением и который возвращает итератор.
<br>При реализации нельзя использовать `zip`, `itertools` или другие сторонние модули.
```python
>>> list(zip(['A', 'B', 'C'], [1, 2, 3]))
    [('A', 1), ('B', 2), ('C', 3)]

>>> list(zip('!', ['A', 'B', 'C', 'D'], range(1, 3)))
    [('!', 'A', 1)]

>>> list(zip('abcd', ['A', 'B', 'C', 'D'], range(0, 40)))
    [('a', 'A', 0), ('b', 'B', 1), ('c', 'C', 2), ('d', 'D', 3)]
```

Используйте что сочтете нужным:
<br>Или класс
```python
class CustomZip:
    pass

list(CustomZip(['A', 'B', 'C'], [1, 2, 3], (22, 33, 44, 55, 66)))
# [('A', 1, 22), ('B', 2, 33), ('C', 3, 44)]
```

Или функцию:
```python
def CustomZip():
    pass

list(custom_zip(['A', 'B', 'C'], [1, 2, 3], (22, 33, 44, 55, 66)))
# [('A', 1, 22), ('B', 2, 33), ('C', 3, 44)]
```

### HW 2.

Найдите все четные числа из списка `VALUES` тремя способами:
* функция, которая возвращает `List[int]`
* функция, которая возвращает `Generator`
* функция, которая возвращает `List[int]`, но используя однострочный list comprehension

```python
from typing import Generator
from typing import List

VALUES = [
    [[10, 11, 12], [13, 14, 15], [16, 17, 18]],
    [[19, 20, 21], [22, 23, 24], [25, 26, 27]],
    [[28, 29, 30], [31, 32, 33], [34, 35, 36]],
]

def get_even_for_loop(values: List) -> List[int]:
    """Return all even numbers using classical for loop.
    :param values: input list of lists with values
    :return: list with int values
    """
    pass

def get_even_for_loop_iterator(values: List) -> Generator:
    """Return all even numbers using classical for loop.
    :param values: input list of lists with values
    :return: generator with int values
    """
    pass

def get_even_list_comprehension(values: List) -> List[int]:
    """Return all even numbers in ONE LINE using list comprehension.
    :param values: input list of lists with values
    :return: list with int values
    """
    pass

print(get_even_for_loop(VALUES))
print(list(get_even_for_loop_iterator(VALUES)))
print(get_even_list_comprehension(VALUES))

# [10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36]
# [10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36]
# [10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36]
```

Не забываем использовать `pylint` + `black` + `isort`

<center>🐍</center>